In [1]:
import json
import urllib
from urllib.request import Request, urlopen
from urllib.parse import urlparse, parse_qs
import pandas as pd
import numpy as np
import googlemaps
from geographiclib.geodesic import Geodesic
from math import atan2, degrees
import math
import re
import time
#지도에 표시하기
import folium
from folium import Marker

In [3]:
#중간 좌표 구하는 함수
def get_optimal_route(start, goal, err_list):
    # waypoint는 최대 5개까지 입력 가능, 
    # 구분자로 |(pipe char) 사용하면 됨(x,y 좌표값으로 넣을 것)
    # waypoint 옵션을 다수 사용할 경우, 아래 함수 포맷을 바꿔서 사용
    
    # naver geocoding service에서 api 가져오기
    client_id = ''
    client_secret = ''
    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={start[0]},{start[1]}&goal={goal[0]},{goal[1]}&option={option}"
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)
    
    response = urllib.request.urlopen(request)
    res = response.getcode()
#     print(res)
    
    if (res == 200):
        response_body = response.read().decode('utf-8')
        if response_body[8] == '0':
            dict_result = json.loads(response_body)
            path_result = dict_result.get("route").get("traoptimal")[0]['path']
            return path_result
        else:
            error_num = response_body[8]
            err_list.append(error_num)
            return 'RESULT ERROR'

    else :
        return 'STATUS ERROR'

In [4]:
#가장 가까운
def get_distance(latitude1, longitude1, latitude2, longitude2):
    earth_radius = 6371  # 지구 반지름(km)
    d_lat = math.radians(latitude2 - latitude1)
    d_lng = math.radians(longitude2 - longitude1)
    a = (math.sin(d_lat / 2) ** 2 +
         math.cos(math.radians(latitude1)) * math.cos(math.radians(latitude2)) *
         math.sin(d_lng / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = earth_radius * c  # 거리(km)
    return distance

In [5]:
#근처 지점 전부 검색
def get_places_nearby(latitude, longitude, radius, keyword):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude},{longitude}",
        "radius": radius,
        "keyword": keyword,
        "key": api_key
    }
    response = requests.get(url, params=params)
    response_json = response.json()
    return response_json["results"]

In [6]:
#가장 가까운 지점 추출
def get_nearest_place(latitude, longitude, radius, keyword):
    places = get_places_nearby(latitude, longitude, radius, keyword)
    if not places:
        return None
    
    min_distance = math.inf
    nearest_place = None
    
    for place in places:
        location = place["geometry"]["location"]
        lat, lng = location["lat"], location["lng"]
        distance = get_distance(latitude, longitude, lat, lng)
        if distance < min_distance:
            min_distance = distance
            nearest_place = location
    
    return nearest_place

In [7]:
def calculate_bearing(lat1, lon1, lat2, lon2):
    x = lon2 - lon1
    y = lat2 - lat1
    bearing = atan2(y, x)
    bearing_degrees = bearing * 180 / math.pi  # 동쪽이 0도인 경우
    return bearing_degrees

In [8]:
def clean_signal(text):
    cleaned = re.sub("[\[\]]", "",str(text))
    return cleaned

In [1]:
#별도의 data 추가
#deg는 출발지, 도착지의 좌표, 주소, 방위각의 data
#route는 출발지에서 도착지에 도착할때까지 차량의 경로 data

deg = pd.read_csv("")
route = pd.read_csv("")

In [12]:
plate = pd.read_csv('C:/traffic/data/서울특별시_서초구_도로안전표지_20221206.csv', encoding = 'cp949')
cctv = pd.read_csv('C:/traffic/data/12_04_08_E_CCTV정보.csv', encoding = 'cp949')
bump = pd.read_csv('C:/traffic/data/12_04_06_E_과속방지턱정보.csv', encoding = 'cp949')

In [13]:
seocho_plate_df = plate[['위도', '경도']]
seocho_cctv_df = cctv[['WGS84위도', 'WGS84경도']]
seocho_bump_df = bump[['WGS84위도', 'WGS84경도']]

In [14]:
seocho_deg = deg[deg['Gu'].str.contains('서초', na = False)]

In [15]:
seocho_acc = route[deg['Gu'].str.contains('서초', na = False)]

In [23]:
#version2    
start_c = [0,0]
plate_c = [0,0]
cctv_c = [0,0]
bump_c = [0,0]

m = folium.Map(location = [37, 127], zoom_start=10)


for i in range(len(seocho_plate_df)):
    
    plate_c[0] = float(seocho_plate_df.iloc[i][0])
    plate_c[1] = float(seocho_plate_df.iloc[i][1])
    
    #도착 지점은 검은색
    folium.Circle(location = plate_c, radius = 7, color='black').add_to(m)
    
for j in range(len(seocho_cctv_df)):
    
    cctv_c[0] = float(seocho_cctv_df.iloc[j][0])
    cctv_c[1] = float(seocho_cctv_df.iloc[j][1])
    
    #도착 지점은 검은색
    folium.Circle(location = cctv_c, radius = 7, color='blue').add_to(m)
    
for n in range(len(seocho_bump_df)):
    
    bump_c[0] = float(seocho_bump_df.iloc[n][0])
    bump_c[1] = float(seocho_bump_df.iloc[n][1])
    
    #도착 지점은 검은색
    folium.Circle(location = bump_c, radius = 7, color='green').add_to(m)
    
    
for l in range(len(seocho_acc)):
    
    first_point_str = clean_signal(seocho_acc.iloc[l][0])
    first_point = first_point_str.split(",")

    first_point_lat = float(first_point[1])
    first_point_lng = float(first_point[0])
    
    #도착 지점은 검은색
    folium.Circle(location = [first_point_lat, first_point_lng], radius = 7, color='red').add_to(m)
    
    

m.save(f"./map/example_v1.html")

In [27]:
#distance = km
dis_num = 0.2
N_no = 0
E_no = 0
W_no = 0
S_no = 0
NE_no = 0
NW_no = 0
SE_no = 0
SW_no = 0

direction_list = ['N', 'E', 'W', 'S', 'NE', 'NW', 'SE', 'SW']

val_list = [N_no, E_no, W_no, S_no, NE_no, NW_no, SE_no, SW_no]


for word in range(len(direction_list)):
    
    direction_word = direction_list[word]
    
    m = folium.Map(location = [37, 127], zoom_start=10)
    
    
    for i in range(len(seocho_plate_df)):
    
        plate_c[0] = float(seocho_plate_df.iloc[i][0])
        plate_c[1] = float(seocho_plate_df.iloc[i][1])
    
        #도착 지점은 검은색
        folium.Circle(location = plate_c, radius = 7, color='black').add_to(m)
    
    for j in range(len(seocho_cctv_df)):

        cctv_c[0] = float(seocho_cctv_df.iloc[j][0])
        cctv_c[1] = float(seocho_cctv_df.iloc[j][1])

        #도착 지점은 검은색
        folium.Circle(location = cctv_c, radius = 7, color='blue').add_to(m)

    for n in range(len(seocho_bump_df)):

        bump_c[0] = float(seocho_bump_df.iloc[n][0])
        bump_c[1] = float(seocho_bump_df.iloc[n][1])

        #도착 지점은 검은색
        folium.Circle(location = bump_c, radius = 7, color='green').add_to(m)

    
    for l in range(len(seocho_deg)):
        if seocho_deg['split'].iloc[l] == direction_word:
            val_list[word] += 1
            first_point_str = clean_signal(seocho_acc.iloc[l][0])
            first_point = first_point_str.split(",")

            first_point_lat = float(first_point[1])
            first_point_lng = float(first_point[0])

            #도착 지점은 검은색
            folium.Circle(location = [first_point_lat, first_point_lng], radius = 7, color='red').add_to(m)


    


    m.save(f"./map/sepcho_{direction_word}.html")
    
print(val_list)

[72, 227, 187, 100, 108, 222, 288, 128]


In [25]:
#seocho 1332
#['N', 'E', 'W', 'S', 'NE', 'NW', 'SE', 'SW']
[72, 227, 187, 100, 108, 222, 288, 128]

[72, 227, 187, 100, 108, 222, 288, 128]

In [34]:
seocho_deg['split'].iloc[0]

'SE'

In [32]:
ex_str = clean_signal(seocho_acc.iloc[l][0])
ex_point = ex_str.split(",")

ex_point_lat = float(ex_point[1])
ex_point_lng = float(ex_point[0])

print(ex_point_lat, ex_point_lng)

37.4448463 127.0571514


In [2]:
import folium
import math

# 시작 좌표 (예: 위도, 경도)
start_lat, start_lon = ex_point_lat, ex_point_lng

# 각도 (예: 북쪽을 향하면 0도, 동쪽을 향하면 90도 등)
if seocho_deg['split'].iloc[0] == 'SE':
    angle = 225

# 벡터의 길이 (위도와 경도의 차이)
length = 0.0003

# 각도를 라디안으로 변환
angle_rad = math.radians(angle)

print(angle_rad)
# 방향성 벡터의 끝점 계산
end_lat = start_lat + length * math.cos(angle_rad)
end_lon = start_lon + length * math.sin(angle_rad)

# 지도 생성
m = folium.Map(location=[start_lat, start_lon], zoom_start=15, tiles="Stamen Toner")

# 시작점과 끝점을 이어 선을 그림
# folium.Circle([(start_lat, start_lon), (end_lat, end_lon)], color="blue").add_to(m)
folium.Circle(location = [start_lat, start_lon], color="red", fill_color="white", fillOpacity = 1, radius = 20).add_to(m)

# 지도 표시
m

In [44]:
color_list = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'cadetblue']
color_list[0]

'red'

In [72]:
#distance = km
dis_num = 0.2
N_no = 0
E_no = 0
W_no = 0
S_no = 0
NE_no = 0
NW_no = 0
SE_no = 0
SW_no = 0

direction_list = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
angle_list = [0, 45, 90, 135, 180, 225, 270, 315]
color_list = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'cadetblue']

val_list = [N_no, E_no, W_no, S_no, NE_no, NW_no, SE_no, SW_no]

# 벡터의 길이 (위도와 경도의 차이)
length = 0.0002

m = folium.Map(location = [37, 127], zoom_start=10)

for word in range(len(direction_list)):
    
    direction_word = direction_list[word]
    
#     if direction_word == 'N':
#         angle = 0
#     elif direction_word == 'NE':
#         angle = 45
#     elif direction_word == 'E':
#         angle = 90
#     elif direction_word == 'SE':
#         angle = 135
#     elif direction_word == 'S':
#         angle = 180
#     elif direction_word == 'SW':
#         angle = 225
#     elif direction_word == 'W':
#         angle = 270
#     elif direction_word == 'NW':
#         angle = 315
        
    angle = angle_list[word]
    color = color_list[word]
    
    # 각도를 라디안으로 변환
    angle_rad = math.radians(angle)
    
    for l in range(len(seocho_deg)):
        if seocho_deg['split'].iloc[l] == direction_word:
            val_list[word] += 1
            first_point_str = clean_signal(seocho_acc.iloc[l][0])
            first_point = first_point_str.split(",")

            first_point_lat = float(first_point[1])
            first_point_lng = float(first_point[0])
            
            end_lat = first_point_lat + length * math.cos(angle_rad)
            end_lon = first_point_lng + length * math.sin(angle_rad)

#             도착 지점은 검은색
#             folium.Circle(location = [first_point_lat, first_point_lng], radius = 7, color='red').add_to(m)
            folium.PolyLine([(first_point_lat, first_point_lng), (end_lat, end_lon)], color= color).add_to(m)

for i in range(len(seocho_plate_df)):
    
    plate_c[0] = float(seocho_plate_df.iloc[i][0])
    plate_c[1] = float(seocho_plate_df.iloc[i][1])

    #도착 지점은 
    folium.Circle(location = plate_c, radius = 5, color='lightgreen').add_to(m)
    
for j in range(len(seocho_cctv_df)):

    cctv_c[0] = float(seocho_cctv_df.iloc[j][0])
    cctv_c[1] = float(seocho_cctv_df.iloc[j][1])

    #도착 지점은 
    folium.Circle(location = cctv_c, radius = 1, color='black').add_to(m)

for n in range(len(seocho_bump_df)):

    bump_c[0] = float(seocho_bump_df.iloc[n][0])
    bump_c[1] = float(seocho_bump_df.iloc[n][1])

    #도착 지점은 
    folium.Circle(location = bump_c, radius = 5, color='darkblue').add_to(m)

    


m.save(f"./map/seocho_all_data_ex2.html")
    
print(val_list)

[72, 108, 227, 288, 100, 128, 187, 222]


# 어느 지역이든 좌표 찾기 코드

In [190]:
#지역명
region = "영등포구"

In [191]:
plate = pd.read_csv('./data/국토교통부_도로안내표지_20210930.csv', encoding = 'cp949')
cctv = pd.read_csv(f'./data/{region}cctv.csv', encoding = 'cp949')
bump = pd.read_csv(f'./data/{region}bump.csv', encoding = 'cp949')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2352\711015229.py:1: DtypeWarning: Columns (2,7,13,14,16,17,23,24,26,27,33,34,36,37,42,43,44,45,46,47,50,52,53,54,55,56,57,58,59,60,62,65,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  plate = pd.read_csv('C:/traffic/data/국토교통부_도로안내표지_20210930.csv', encoding = 'cp949')


In [192]:
print(len(plate))
print(len(cctv))
print(len(bump))

219211
4397
1801


In [193]:
region_cctv_df = cctv[['WGS84위도', 'WGS84경도']]
region_bump_df = bump[['WGS84위도', 'WGS84경도']]

In [194]:
region_deg = deg[deg['Gu'].str.contains(region, na = False)]
region_acc = route[deg['Gu'].str.contains(region, na = False)]

In [195]:
region_plate_del = plate[plate['소재지도로명주소'].str.contains(region, na = False)]
region_plate_df = region_plate_del[['위도', '경도']]
region_plate_df = region_plate_df.dropna(axis=0)

In [188]:
#distance = km
dis_num = 0.2
N_no = 0
E_no = 0
W_no = 0
S_no = 0
NE_no = 0
NW_no = 0
SE_no = 0
SW_no = 0

direction_list = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
angle_list = [0, 45, 90, 135, 180, 225, 270, 315]
color_list = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'cadetblue']

val_list = [N_no, E_no, W_no, S_no, NE_no, NW_no, SE_no, SW_no]

# 벡터의 길이 (위도와 경도의 차이)
length = 0.0002

m = folium.Map(location = [37, 127], zoom_start=10)

for word in range(len(direction_list)):
    
    direction_word = direction_list[word]
    
#     if direction_word == 'N':
#         angle = 0
#     elif direction_word == 'NE':
#         angle = 45
#     elif direction_word == 'E':
#         angle = 90
#     elif direction_word == 'SE':
#         angle = 135
#     elif direction_word == 'S':
#         angle = 180
#     elif direction_word == 'SW':
#         angle = 225
#     elif direction_word == 'W':
#         angle = 270
#     elif direction_word == 'NW':
#         angle = 315
        
    angle = angle_list[word]
    color = color_list[word]
    
    # 각도를 라디안으로 변환
    angle_rad = math.radians(angle)
    
    for l in range(len(region_deg)):
        if region_deg['split'].iloc[l] == direction_word:
            val_list[word] += 1
            first_point_str = clean_signal(region_acc.iloc[l][0])
            first_point = first_point_str.split(",")

            first_point_lat = float(first_point[1])
            first_point_lng = float(first_point[0])
            
            end_lat = first_point_lat + length * math.cos(angle_rad)
            end_lon = first_point_lng + length * math.sin(angle_rad)

#             도착 지점은 검은색
#             folium.Circle(location = [first_point_lat, first_point_lng], radius = 7, color='red').add_to(m)
            folium.PolyLine([(first_point_lat, first_point_lng), (end_lat, end_lon)], color= color).add_to(m)

for i in range(len(region_plate_df)):
    
    plate_c[0] = float(region_plate_df.iloc[i][0])
    plate_c[1] = float(region_plate_df.iloc[i][1])

    #도착 지점은 
    folium.Circle(location = plate_c, radius = 2, color='lightgreen').add_to(m)
    
for j in range(len(region_cctv_df)):

    cctv_c[0] = float(region_cctv_df.iloc[j][0])
    cctv_c[1] = float(region_cctv_df.iloc[j][1])

    #도착 지점은 
    folium.Circle(location = cctv_c, radius = 2, color='black').add_to(m)

for n in range(len(region_bump_df)):

    bump_c[0] = float(region_bump_df.iloc[n][0])
    bump_c[1] = float(region_bump_df.iloc[n][1])

    #도착 지점은 
    folium.Circle(location = bump_c, radius = 2, color='darkblue').add_to(m)

    


m.save(f"./map/{region}_all_data_ex2.html")
    
print(val_list)

[72, 108, 227, 288, 100, 128, 187, 222]


In [197]:
#distance = km
dis_num = 0.2
N_no = 0
E_no = 0
W_no = 0
S_no = 0
NE_no = 0
NW_no = 0
SE_no = 0
SW_no = 0

direction_list = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
angle_list = [0, 45, 90, 135, 180, 225, 270, 315]
color_list = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'cadetblue']

val_list = [N_no, E_no, W_no, S_no, NE_no, NW_no, SE_no, SW_no]

# 벡터의 길이 (위도와 경도의 차이)
length = 0.0002

m = folium.Map(location = [37, 127], zoom_start=10, tiles="Stamen Toner")

for word in range(len(direction_list)):
    
    direction_word = direction_list[word]
    
#     if direction_word == 'N':
#         angle = 0
#     elif direction_word == 'NE':
#         angle = 45
#     elif direction_word == 'E':
#         angle = 90
#     elif direction_word == 'SE':
#         angle = 135
#     elif direction_word == 'S':
#         angle = 180
#     elif direction_word == 'SW':
#         angle = 225
#     elif direction_word == 'W':
#         angle = 270
#     elif direction_word == 'NW':
#         angle = 315
        
    angle = angle_list[word]
    color = color_list[word]
    
    # 각도를 라디안으로 변환
    angle_rad = math.radians(angle)
    
    for l in range(len(region_deg)):
        if region_deg['split'].iloc[l] == direction_word:
            val_list[word] += 1
            first_point_str = clean_signal(region_acc.iloc[l][0])
            first_point = first_point_str.split(",")

            first_point_lat = float(first_point[1])
            first_point_lng = float(first_point[0])
            
            end_lat = first_point_lat + length * math.cos(angle_rad)
            end_lon = first_point_lng + length * math.sin(angle_rad)

#             도착 지점은 검은색
            folium.Circle(location = [first_point_lat, first_point_lng], radius = 4, color = color).add_to(m)
#             folium.PolyLine([(first_point_lat, first_point_lng), (end_lat, end_lon)], color= color).add_to(m)

for i in range(len(region_plate_df)):
    
    plate_c[0] = float(region_plate_df.iloc[i][0])
    plate_c[1] = float(region_plate_df.iloc[i][1])

    #도착 지점은 
    folium.Circle(location = plate_c, color="black", fill_color="red", fillOpacity = 1, radius = 4).add_to(m)
    
for j in range(len(region_cctv_df)):

    cctv_c[0] = float(region_cctv_df.iloc[j][0])
    cctv_c[1] = float(region_cctv_df.iloc[j][1])

    #도착 지점은 
    folium.Circle(location = cctv_c, color="black", fill_color="gray", fillOpacity = 1, radius = 4).add_to(m)

for n in range(len(region_bump_df)):

    bump_c[0] = float(region_bump_df.iloc[n][0])
    bump_c[1] = float(region_bump_df.iloc[n][1])

    #도착 지점은 
    folium.Circle(location = bump_c, color="black", fill_color="yellow", fillOpacity = 1, radius = 4).add_to(m)

    


m.save(f"./map/{region}_image_ex3.html")
    
print(val_list)

[214, 272, 304, 361, 207, 327, 348, 395]


In [1]:
m = folium.Map(location = [37, 127], zoom_start=10, tiles = 'cartodbpositron')

for n in range(len(seocho_bump_df)):

    bump_c[0] = float(seocho_bump_df.iloc[n][0])
    bump_c[1] = float(seocho_bump_df.iloc[n][1])

    #도착 지점은 검은색
    folium.Circle(location = bump_c, radius = 7, color='green').add_to(m)

m

In [34]:
#version1
start_lat = deg['ori_lat']
start_lon = deg['ori_lon']
goal_lat = deg['des_lat']
goal_lon = deg['des_lon']
        
start_c = [0,0]
goal_c = [0,0]
    
# for i in range(len(comp)):
for i in range(19,20):
    if comp['comp'][i] == 0:
        
        start_c[1] = start_lon[i]
        start_c[0] = start_lat[i]

        goal_c[1] = goal_lon[i]
        goal_c[0] = goal_lat[i]
    
        mid = int(index_num[i])//2
        ex_mid = route.iloc[i][mid]

        new_ex_mid = clean_signal(ex_mid)
        ex_mid_final = new_ex_mid.split(",")

        ex_mid_lat = ex_mid_final[1]
        ex_mid_lng = ex_mid_final[0]
        
        m = folium.Map(location = [ex_mid_lat, ex_mid_lng], zoom_start=16)
        
        for j in range(len(route_drop.iloc[0])):
            isNaN = pd.isna(route_drop.iloc[i][j])
            
#             print(isNaN)
            
            if isNaN == False:
                ex = route.iloc[i][j]

                new_ex = clean_signal(ex)
                ex_final = new_ex.split(",")

                ex_lat = ex_final[1]
                ex_lng = ex_final[0]

#                 print(ex_lat)
#                 print(ex_lng)

                folium.Circle(location = [ex_lat, ex_lng], radius = 7, icon=folium.Icon(color='blue',icon='star')).add_to(m)
    
#         #사고 지점은 빨간색
#         Marker(location = start_c,
#                    icon=folium.Icon(color='red',icon='star')
#                   ).add_to(m)

        #도착 지점은 검은색
        Marker(location = goal_c,
                   icon=folium.Icon(color='black',icon='star')
                  ).add_to(m)
      
        m.save(f"./map/example_ver1_{i}.html")

In [170]:
#version2
start_lat = deg['ori_lat']
start_lon = deg['ori_lon']
goal_lat = deg['des_lat']
goal_lon = deg['des_lon']
        
start_c = [0,0]
goal_c = [0,0]

one_two_point = []
    
# for i in range(len(comp)):
for i in range(0,20):
    one_two_point = []
    if comp['comp'][i] == 0:
        
        start_c[1] = start_lon[i]
        start_c[0] = start_lat[i]

        goal_c[1] = goal_lon[i]
        goal_c[0] = goal_lat[i]
    
        mid = int(index_num[i])//2
        ex_mid = route.iloc[i][mid]

        new_ex_mid = clean_signal(ex_mid)
        ex_mid_final = new_ex_mid.split(",")

        ex_mid_lat = ex_mid_final[1]
        ex_mid_lng = ex_mid_final[0]
        
        m = folium.Map(location = [ex_mid_lat, ex_mid_lng], zoom_start=16)
        
        for j in range(0,2):
            isNaN = pd.isna(route_drop.iloc[i][j])
            
#             print(isNaN)
            
            if isNaN == False:
                ex = route.iloc[i][j]

                new_ex = clean_signal(ex)
                ex_final = new_ex.split(",")

                ex_lat = ex_final[1]
                ex_lng = ex_final[0]
                
                one_two_point.append([float(ex_lat), float(ex_lng)])

#                 print(ex_lat)
#                 print(ex_lng)

        Marker(location = one_two_point[0],icon=folium.Icon(color='green',icon='home')).add_to(m)
        Marker(location = one_two_point[1],icon=folium.Icon(color='green',icon='flag')).add_to(m)

        print(one_two_point)

        folium.PolyLine(locations = one_two_point, tooltip = 'Polyline', color='green').add_to(m)
    
        #사고 지점은 빨간색
        Marker(location = start_c,
                   icon=folium.Icon(color='red',icon='home')
                  ).add_to(m)

        #도착 지점은 검은색
        Marker(location = goal_c,
                   icon=folium.Icon(color='red',icon='flag')
                  ).add_to(m)
        
        folium.PolyLine(locations = [start_c, goal_c], tooltip = 'Polyline', color='red').add_to(m)
      
        m.save(f"./map/example_ver2_{i}.html")
    


[[37.564829, 126.996462], [37.56493, 126.9962553]]
[[37.5662666, 127.0008934], [37.5662927, 127.0015364]]
[[37.555805, 127.0087152], [37.5558662, 127.0086865]]
[[37.5594086, 127.0132839], [37.5595441, 127.0133645]]
[[37.5694823, 127.0001846], [37.5694731, 127.0001416]]
[[37.5619326, 127.0040247], [37.5620317, 127.0040127]]
[[37.558827, 127.002202], [37.5588333, 127.0022122]]
[[37.5706102, 127.0094978], [37.5707227, 127.0094507]]
[[37.5681154, 127.0209432], [37.5682497, 127.0209367]]
[[37.5683215, 127.0066536], [37.5694616, 127.0066144]]


In [191]:
#version3
start_lat = deg['ori_lat']
start_lon = deg['ori_lon']
goal_lat = deg['des_lat']
goal_lon = deg['des_lon']
        
start_c = [0,0]
goal_c = [0,0]

one_two_point = []
    
# for i in range(len(comp)):
for i in range(0,300):
    one_two_point = []
    if comp['comp'][i] == 0:
        
        start_c[1] = start_lon[i]
        start_c[0] = start_lat[i]

        goal_c[1] = goal_lon[i]
        goal_c[0] = goal_lat[i]
    
        mid = int(index_num[i])//2
        ex_mid = route.iloc[i][mid]

        new_ex_mid = clean_signal(ex_mid)
        ex_mid_final = new_ex_mid.split(",")

        ex_mid_lat = ex_mid_final[1]
        ex_mid_lng = ex_mid_final[0]
        
        m = folium.Map(location = [ex_mid_lat, ex_mid_lng], zoom_start=16)
        
        for j in range(len(route_drop.iloc[0])):
            num_j = j%5
            
            isNaN = pd.isna(route_drop.iloc[i][j])
            
#             print(isNaN)
            
            if isNaN == False:
                ex = route.iloc[i][j]

                new_ex = clean_signal(ex)
                ex_final = new_ex.split(",")

                ex_lat = ex_final[1]
                ex_lng = ex_final[0]
                
                if j == 0 or j == 1:
                    one_two_point.append([float(ex_lat), float(ex_lng)])
                elif num_j == 0:
#                     print("5")
                    Marker(location = [ex_lat, ex_lng],icon=folium.Icon(color='blue',icon='star')).add_to(m)
                    
#                 print(ex_lat)
#                 print(ex_lng)

        Marker(location = one_two_point[0],icon=folium.Icon(color='green',icon='home')).add_to(m)
        Marker(location = one_two_point[1],icon=folium.Icon(color='green',icon='flag')).add_to(m)

#         print(one_two_point)

        folium.PolyLine(locations = one_two_point, tooltip = 'Polyline', color='green').add_to(m)
    
        #사고 지점은 빨간색
        Marker(location = start_c,
                   icon=folium.Icon(color='red',icon='home')
                  ).add_to(m)

        #도착 지점은 검은색
        Marker(location = goal_c,
                   icon=folium.Icon(color='red',icon='flag')
                  ).add_to(m)
        
        folium.PolyLine(locations = [start_c, goal_c], tooltip = 'Polyline', color='red').add_to(m)
      
        m.save(f"./map/example_ver3_{i}.html")

In [116]:
num = 0

for j in range(len(route_drop.iloc[0])):
    print(num)
    num += 1
    isNaN = pd.isna(route_drop.iloc[0][j])

    print(isNaN)

0
False
1
False
2
False
3
False
4
False
5
False
6
False
7
False
8
False
9
False
10
False
11
False
12
False
13
False
14
False
15
False
16
False
17
False
18
False
19
False
20
False
21
False
22
False
23
False
24
False
25
False
26
False
27
False
28
False
29
False
30
False
31
False
32
False
33
False
34
False
35
False
36
False
37
False
38
False
39
False
40
False
41
False
42
False
43
False
44
False
45
False
46
False
47
False
48
False
49
False
50
False
51
False
52
False
53
False
54
False
55
False
56
False
57
False
58
False
59
False
60
False
61
False
62
False
63
False
64
False
65
False
66
False
67
False
68
False
69
False
70
False
71
False
72
False
73
False
74
False
75
False
76
False
77
False
78
False
79
False
80
False
81
False
82
False
83
False
84
False
85
False
86
False
87
False
88
False
89
True
90
True
91
True
92
True
93
True
94
True
95
True
96
True
97
True
98
True
99
True
100
True
101
True
102
True
103
True
104
True
105
True
106
True
107
True
108
True
109
True
110
True
111
True
112
True
113
